In [14]:
!pip install -q google-generativeai


In [15]:
import google.generativeai as genai
import getpass
import sympy as sp
import re

api_key = getpass.getpass("Enter your Gemini API key: ")
genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-1.5-flash-latest")

def is_numerical_math_question(text):
    # Checks if it's a math or numerical problem (simple heuristic)
    return bool(re.search(r"\d", text)) and any(op in text for op in ['+', '-', '*', '/', '=', '^', '**', 'solve', 'find', 'calculate'])

def solve_math_steps(text):
    try:
        match = re.search(r'([-+*/^=xX0-9\(\)a-zA-Z\s\.]+=[-+*/^xX0-9\(\)\s\.a-zA-Z]+)', text)
        if not match:
            return "Couldn't find a valid equation."

        equation_str = match.group().replace(" ", "")
        var_match = re.search(r"[a-zA-Z]", equation_str)
        if not var_match:
            return "Could not detect variable."

        var = sp.symbols(var_match.group())
        lhs_str, rhs_str = equation_str.split("=")
        lhs = sp.sympify(lhs_str)
        rhs = sp.sympify(rhs_str)
        eq = sp.Eq(lhs, rhs)

        steps = []
        steps.append(f"Step 1: Original Equation → {lhs} = {rhs}")

        combined = lhs - rhs
        steps.append(f"Step 2: Bring all terms to one side → {combined} = 0")
        steps.append("Explanation: Moved all terms to one side.")

        simplified = sp.simplify(combined)
        steps.append(f"Step 3: Simplify → {simplified} = 0")
        steps.append("Explanation: Combined like terms.")

        sol = sp.solve(eq, var)
        steps.append(f"Step 4: Solve for {var} → {var} = {sol[0]}")
        steps.append("Explanation: Isolated the variable.")

        return "\n".join(steps)

    except Exception as e:
        return f"Math Solver Error: {str(e)}"

def ask_gemini_stepwise(prompt):
    try:
        stepwise_prompt = f"""Answer this question step-by-step in a clear, numbered format.

        Question: {prompt}

        Start the answer like:
        Step 1: ...
        Step 2: ...
        ...
        """
        response = model.generate_content(stepwise_prompt)
        return response.text
    except Exception as e:
        return f"Gemini error: {str(e)}"

def homework_helper_agent(user_input):
    if is_numerical_math_question(user_input.lower()):
        return solve_math_steps(user_input)
    else:
        return ask_gemini_stepwise(user_input)


Enter your Gemini API key: ··········


In [22]:
question = input("Ask your homework question: ")
print(homework_helper_agent(question))

Ask your homework question: Solve for x: 2*x + 5 = 15
Step 1: Original Equation → 2*x + 5 = 15
Step 2: Bring all terms to one side → 2*x - 10 = 0
Explanation: Moved all terms to one side.
Step 3: Simplify → 2*x - 10 = 0
Explanation: Combined like terms.
Step 4: Solve for x → x = 5
Explanation: Isolated the variable.


In [21]:
question = input("Ask your homework question: ")
print(homework_helper_agent(question))

Ask your homework question: Calculate the kinetic energy of a 2 kg ball moving at 3 m/s.
Couldn't find a valid equation.


In [23]:
question = input("Ask your homework question: ")
print(homework_helper_agent(question))

Ask your homework question: Explain the causes of the French Revolution.
Step 1: **Enlightenment Ideals:** The Enlightenment, a philosophical movement emphasizing reason, individual liberty, and popular sovereignty, profoundly influenced French thinkers and revolutionaries.  Ideas of natural rights (life, liberty, and property), popular sovereignty (government by the consent of the governed), and separation of powers fueled discontent with the absolute monarchy.

Step 2: **Economic Crisis:** France faced severe financial problems stemming from decades of extravagant spending by the monarchy, costly involvement in wars (like the American Revolution), and a rigid tax system that disproportionately burdened the peasantry and the Third Estate.  Bad harvests led to food shortages and soaring bread prices, exacerbating poverty and resentment.

Step 3: **Social Inequality:** French society was rigidly divided into three Estates: the First Estate (clergy), the Second Estate (nobility), and the